<a href="https://colab.research.google.com/github/Justmin-Lee/ML_Lecture/blob/main/4_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
tf.set_random_seed(777)  # for reproducibility

In [2]:
# load file on my desktop
from google.colab import files
myfile = files.upload()

Saving data-01-test-score.csv to data-01-test-score.csv


In [5]:
# Queue Runners
filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. 
# Also specifies the type of the decoded result.
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

In [10]:
# collect batches of csv in
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

In [11]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [12]:
# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [13]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
0 Cost:  2571.649 
Prediction:
 [[135.61624]
 [ 73.14668]
 [129.85417]
 [117.99579]
 [135.61624]
 [ 96.54288]
 [ 69.79834]
 [100.46828]
 [ 73.14668]
 [117.17079]]
10 Cost:  8.272571 
Prediction:
 [[192.27115]
 [154.38324]
 [185.08638]
 [182.7635 ]
 [198.50995]
 [141.23898]
 [102.76999]
 [147.55731]
 [108.07028]
 [171.96458]]
20 Cost:  8.367167 
Prediction:
 [[192.6867 ]
 [154.71048]
 [185.49286]
 [183.1566 ]
 [198.9458 ]
 [141.54587]
 [103.00974]
 [147.89139]
 [108.32851]
 [172.357  ]]
30 Cost:  8.333593 
Prediction:
 [[192.68571]
 [154.7034 ]
 [185.49739]
 [183.1537 ]
 [198.95035]
 [141.54619]
 [103.0234 ]
 [147.90262]
 [108.34736]
 [172.37367]]
40 Cost:  8.298596 
Prediction:
 [[192.68188]
 [154.69408]
 [185.49915]
 [183.14815]
 [198.95198]
 [141.54442]
 [103.03547]
 [147.9116 ]
 [108.36453]
 [172.38771]]
50 Cost:  8.263895 
Prediction:
 [[192.67807]
 [154.68483]
 [185.5009 ]
 [183.14261]
 [198.9536 ]


In [14]:
# Start parpulating the filename queue
coord.request_stop()
coord.join(threads)

# Ask my score
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))


Your score will be  [[185.6457]]
Other scores will be  [[168.41559]
 [179.75922]]
